# Import Packages

In [1]:
import pandas as pd
import pulp
from sklearn.metrics.pairwise import manhattan_distances
import matplotlib.pyplot as plt

# Import data and define main parameters

In [2]:
locations = pd.read_csv('locations.csv')
locations_list = locations['Location'].unique()
n_locations = len(locations_list)
destinations_list = locations_list[1:]
locations = locations.set_index(['Location'])

In [3]:
locations.head()

,X Axis,Y Axis
Location,,
0,456,320
1,228,0
2,912,0
3,0,80
4,114,80


In [4]:
distances = pd.DataFrame(manhattan_distances(locations.values, locations.values, sum_over_features=True))

In [5]:
vehicles = [1,2,3]
n_vehicles = len(vehicles)

# Write the problem in PuLP

## Definition of the variables

In [6]:

x = pulp.LpVariable.dicts("x",
                          ((i,j,k) for i in locations_list for j in locations_list for k in vehicles),
                          cat='Binary')

u = pulp.LpVariable.dicts("u",((i) for i in locations_list),
                          lowBound=0)



## Type and name of the problem

In [7]:
model = pulp.LpProblem("Vehicle routing problem", pulp.LpMinimize)

## Definition of the Objective Function

In [8]:
model += pulp.lpSum([x[(i,j,k)]*distances.loc[i,j] for i in locations_list for j in locations_list for k in vehicles])

## Definition of constraints

In [ ]:
#1 - #2 destinations are visited only once

for i in destinations_list:
    model += pulp.lpSum(x[(i,j,k)] for j in locations_list for k in vehicles) == 1

for j in destinations_list:
    model += pulp.lpSum(x[(i, j, k)] for i in locations_list for k in vehicles) == 1

#3 the number of vehicles that leave the origin is the same than the number that arrives

model += pulp.lpSum(x[(0,j,k)] for j in locations_list for k in vehicles) == n_vehicles

model += pulp.lpSum(x[(i,0,k)] for i in locations_list for k in vehicles) == n_vehicles

#4 no subtours allowed

for k in vehicles:
    for i in locations_list:
        for j in destinations_list:
            model += u[i]-u[j]+n_locations*x[(i,j,k)] <= n_locations-1


#each destination is visited only once

for k in vehicles:
    for j in destinations_list:
        model += pulp.lpSum(x[(i,j,k)] for i in locations_list) == pulp.lpSum(x[(j,i,k)] for i in locations_list)



# Set a capacity of maximum number of nodes that can be visited by a vehicle
capacity = 7
for k in vehicles:
    model += pulp.lpSum(x[(i,j,k)] for i in locations_list for j in destinations_list) <= capacity

## Solve the model

In [ ]:
#model.solve()
model.solve(pulp.GLPK_CMD(options=['--mipgap', '0.01']))
#model.solve(solver=pulp.GUROBI(MIPGap = 0.01))

In [ ]:
print(pulp.LpStatus[model.status])

In [ ]:
# Print our objective function value (Total Distance)
print("The value of the objective function is: "+ str(pulp.value(model.objective)))

# Wrangle the solution

In [ ]:
output_x=[]

for i,j,k in x:
    x_output = {
        'vehicle':k,
        'start': i,
        'end':j,
        'status':x[(i,j,k)].varValue
    }
    output_x.append(x_output)

In [ ]:
x_output_df = pd.DataFrame(output_x)

In [ ]:
x_output_df = x_output_df[['vehicle','start','end','status']]

In [ ]:
x_output_df = x_output_df.set_index(['vehicle','start','end'])

In [ ]:
x_output_df = x_output_df.sort_index()

In [ ]:
x_output_df.head()

In [ ]:
x_active_output = x_output_df[x_output_df['status'] == 1]

In [ ]:
x_active_origins = x_active_output.reset_index()
x_active_origins = x_active_origins.set_index(['vehicle','start'])

In [ ]:
routes={}


for k in vehicles:
    origin = 0
    route = []
    route.append(origin)
    while True:
        destination = int(x_active_origins.loc[(k, origin), 'end'])
        route.append(destination)
        origin = destination
        if destination == 0:
            break
    routes[k] = route


# List the routes for each vehicle

In [ ]:
routes

# Visualize the solution

In [ ]:
for k in vehicles:
    selected_route = locations.loc[routes[k],:]
    x_values = list(selected_route['X Axis'])
    y_values = list(selected_route['Y Axis'])
    plt.plot(x_values,y_values,'.',linestyle='dashed')

x_coordinates = list(locations['X Axis'])
y_coordinates = list(locations['Y Axis'])

for i in locations_list:
    label = locations_list[i]
    coord = (x_coordinates[i],y_coordinates[i])

    plt.annotate(label,
                 coord,
                 textcoords="offset points",
                 xytext=(0,10),
                 ha='center')

plt.grid(b=None,which='both',axis='both')
plt.legend(vehicles, loc='upper left')

plt.show()